This file used API key from openrouteservice to get coordinates for each properties in the rental dataset.
PTV dataset, healthcare, and groceries dataset then merged with the rental dataset to create proximity features for each properties

In [ ]:
import pandas as pd
import geopandas as gpd
import openrouteservice as ors
import folium
import sys
import concurrent.futures
from tqdm import tqdm
from threading import Semaphore
import json
import folium
import openrouteservice
import json
import time
import openrouteservice
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon
from shapely.geometry import shape
from geopy.distance import geodesic

from shapely import wkt
from openrouteservice.exceptions import ApiError
sys.path.append('../scripts')
from utilities import get_vic_subs_as_csv

In [2]:
client = ors.Client(key='YOUR API KEY HERE')


Code to get geometry of grocery stores

In [3]:
def convert_to_centroid(geometry):
    geom = wkt.loads(geometry)
    if isinstance(geom, Polygon):
        return geom.centroid.wkt 
    return geometry  

df = pd.read_csv('../data/raw/grocery_markets_victoria.csv')

df['geometry'] = df['geometry'].apply(convert_to_centroid)

df.to_csv('../data/raw/grocery_markets_victoria_processed.csv', index=False)

print("CSV file has been processed and saved as 'processed_file.csv'.")

CSV file has been processed and saved as 'processed_file.csv'.


In [4]:
input_file = '../data/curated/preprocessed_current_rent.csv'
output_file = '../data/curated/house_transport_part_1.csv'

    

Function for getting coordinates of each house based on its address

In [5]:
RATE_LIMIT = 100  
REQUEST_INTERVAL = 60 / RATE_LIMIT 

semaphore = Semaphore(3)  
def get_coordinates(address):
    try:
        with semaphore:
            response = client.pelias_search(address)
            if response['features']:
                coords = response['features'][0]['geometry']['coordinates']
                return {'latitude': coords[1], 'longitude': coords[0]}
            else:
                return {'latitude': None, 'longitude': None}
    except Exception as e:
        print(f"Error fetching coordinates for address '{address}': {e}")
        return (coords[1], coords[0])

def worker(row):
    address = row.get('address') 
    if address:
        coords = get_coordinates(address)
        return coords
    return None

In [6]:

df = pd.read_csv(input_file)

df['geometry'] = None

In [7]:
max_threads = 3

with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    futures = {executor.submit(worker, row): index for index, row in list(df.iterrows())[:5000]}

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing items"):
        index = futures[future]
        try:
            df.at[index, 'geometry'] = future.result()
        except Exception as e:
            print(f"An error occurred at index {index}: {e}")


Processing items:  10%|▉         | 484/4987 [04:38<23:04,  3.25it/s]  /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
Processing items:  10%|▉         | 486/4987 [04:42<1:05:33,  1.14it/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
Processing items:  10%|█         | 503/4987 [04:53<1:11:

In [8]:
df.to_csv(output_file, index=False)

print(f"Updated data with coordinates saved to {output_file}")

Updated data with coordinates saved to ../data/curated/house_transport_part_2.csv


In [ ]:
client = ors.Client(key='YOUR API KEY HERE')


In [ ]:
input_file = '../data/curated/preprocessed_current_rent.csv'
output_file = '../data/curated/house_transport_part_2.csv'

In [ ]:
max_threads = 3

with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    futures = {executor.submit(worker, row): index for index, row in list(df.iterrows())[5000:]}

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing items"):
        index = futures[future]
        try:
            df.at[index, 'geometry'] = future.result()
        except Exception as e:
            print(f"An error occurred at index {index}: {e}")

In [ ]:
df.to_csv(output_file, index=False)

print(f"Updated data with coordinates saved to {output_file}")

In [9]:
housing_file_1 = '../data/curated/house_transport_part_2.csv'
housing_file_2 = '../data/curated/house_transport_part_1.csv'
df1 = pd.read_csv(housing_file_1)
df2 = pd.read_csv(housing_file_2)

df1['geometry'] = df1['geometry'].combine_first(df2['geometry'])
merged_file = '../data/curated/merged_houses_with_complete_geometry.csv'
df1.to_csv(merged_file, index=False)

print(f"Merged file saved to: {merged_file}")

Merged file saved to: ../data/curated/merged_houses_with_complete_geometry.csv


In [23]:
houses_coordinates = []

for index, row in df1.iterrows():
    geometry = row['geometry']
    
    if isinstance(geometry, str):
        try:
            geometry_dict = eval(geometry)
            
            if 'latitude' in geometry_dict and 'longitude' in geometry_dict:
                latitude = geometry_dict['latitude']
                longitude = geometry_dict['longitude']
                houses_coordinates.append({
                    'address': row['address'],
                    'coordinates': (latitude, longitude)
                })
            else:
                print(f"Missing latitude or longitude for house: {row['address']}, geometry: {geometry}")
        
        except Exception as e:
            print(f"Error parsing geometry for house: {row['address']}, error: {e}")
    else:
        print(f"Invalid geometry format for house: {row['address']}, geometry: {geometry}")

# Now houses_coordinates contains all the valid house addresses and their coordinates


In [ ]:
houses_coordinates

Function to calculate closest stations

In [10]:
gdf_regional = gpd.read_file("../data/PTV/", layer='PTV_REGIONAL_TRAIN_STATION')
gdf_regional = gdf_regional.to_crs(epsg=4326)

gdf_metro = gpd.read_file("../data/PTV/", layer='PTV_METRO_TRAIN_STATION')
gdf_metro = gdf_metro.to_crs(epsg=4326)

gdf_tram = gpd.read_file("../data/PTV_tram_bus/", layer='PTV_METRO_TRAM_STOP')
gdf_tram = gdf_tram.to_crs(epsg=4326)


In [11]:
gdf_tram = gpd.read_file("../data/PTV_tram_bus/", layer='PTV_METRO_TRAM_STOP')
gdf_tram = gdf_tram.to_crs(epsg=4326)




In [12]:
def prepare_station_data(gdf):
    return gdf[['geometry', 'STOP_ID', 'STOP_NAME']].apply(
        lambda row: ((row['geometry'].y, row['geometry'].x), row['STOP_ID'], row['STOP_NAME']), axis=1
    ).tolist()
    
train_station_gdf = gpd.GeoDataFrame(pd.concat([gdf_metro, gdf_regional], ignore_index=True))
tram_station_gdf = gpd.GeoDataFrame(gdf_tram)

train_station_data = prepare_station_data(train_station_gdf)
tram_station_data = prepare_station_data(tram_station_gdf)

In [13]:
train_station_gdf = gpd.GeoDataFrame(pd.concat([gdf_metro, gdf_regional], ignore_index=True))
tram_station_gdf = gpd.GeoDataFrame(gdf_tram)

train_station_data = prepare_station_data(train_station_gdf)
tram_station_data = prepare_station_data(tram_station_gdf)


In [14]:
housing_file = '../data/curated/merged_houses_with_complete_geometry.csv'
houses_df = pd.read_csv(housing_file)

# Convert 'geometry' column to a usable format
houses_df['geometry'] = houses_df['geometry'].apply(eval) 


In [15]:
houses_df

,address,parking,type,num_schools,cost,suburb,beds,baths,cost/(beds+baths),geometry
0,"1302/1 Powlett Street, East Melbourne VIC 3002",2,Apartment / Unit / Flat,12,1795.0,East Melbourne,3,2,359.000000,"{'latitude': -37.816107, 'longitude': 144.984947}"
1,"109/150 Clarendon Street, East Melbourne VIC 3002",2,Apartment / Unit / Flat,12,1595.0,East Melbourne,2,2,398.750000,"{'latitude': -37.812236, 'longitude': 144.983539}"
2,"15 Berry Street, East Melbourne VIC 3002",2,House,13,2600.0,East Melbourne,4,3,371.428571,"{'latitude': -37.818765, 'longitude': 144.988428}"
3,"8.15/280 Albert Street, East Melbourne VIC 3002",1,Apartment / Unit / Flat,13,1450.0,East Melbourne,2,2,362.500000,"{'latitude': -37.809741, 'longitude': 144.981764}"
4,"21/322 Albert St, East Melbourne VIC 3002",1,Apartment / Unit / Flat,12,1100.0,East Melbourne,2,2,275.000000,"{'latitude': -37.80984, 'longitude': 144.980274}"
...,...,...,...,...,...,...,...,...,...,...
9982,"13 Nunkeri Court, Clyde North VIC 3978",1,House,4,550.0,Clyde North,3,2,110.000000,"{'latitude': -38.149821, 'longitude': 145.354085}"
9983,"22 Pleven Rise, Clyde North VIC 3978",2,House,4,520.0,Clyde North,4,2,86.666667,"{'latitude': -38.149821, 'longitude': 145.354085}"
9984,"30 Haflinger Avenue, Clyde North VIC 3978",1,House,4,520.0,Clyde North,3,2,104.000000,"{'latitude': -38.149821, 'longitude': 145.354085}"
9985,"16 Nokota Lane, Clyde North VIC 3978",0,House,4,490.0,Clyde North,3,2,98.000000,"{'latitude': -38.149821, 'longitude': 145.354085}"


In [16]:
def find_closest_station(house_coords, stations):
    closest_station = None
    min_distance = float('inf')
    closest_station_id = None
    closest_station_name = None
    
    for (station_coords, station_id, station_name) in stations:
        distance = geodesic(house_coords, station_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_station = station_coords
            closest_station_id = station_id
            closest_station_name = station_name
            
    return closest_station, closest_station_id, closest_station_name, min_distance

In [17]:
for idx, row in houses_df.iterrows():
    house_coords = (row['geometry']['latitude'], row['geometry']['longitude'])
    
    # Find the closest train station
    closest_train_station_coords, closest_train_station_id, closest_train_station_name, train_distance = find_closest_station(house_coords, train_station_data)
    
    # Find the closest tram station
    closest_tram_station_coords, closest_tram_station_id, closest_tram_station_name, tram_distance = find_closest_station(house_coords, tram_station_data)
    
    houses_df.at[idx, 'closest_train_station_name'] = closest_train_station_name
    houses_df.at[idx, 'closest_train_station_distance_km'] = train_distance
    houses_df.at[idx, 'closest_tram_station_name'] = closest_tram_station_name
    houses_df.at[idx, 'closest_tram_station_distance_km'] = tram_distance
    
    # Print progress every 1000 houses
    if (idx + 1) % 1000 == 0:
        print(f"Processed {idx + 1} houses...")

output_file = '../data/curated/houses_with_closest_stations.csv'
houses_df.to_csv(output_file, index=False)

print(f"Merged data with closest stations saved to: {output_file}")

Processed 1000 houses...
Processed 2000 houses...
Processed 3000 houses...
Processed 4000 houses...
Processed 5000 houses...
Processed 6000 houses...
Processed 7000 houses...
Processed 8000 houses...
Processed 9000 houses...
Merged data with closest stations saved to: ../data/curated/houses_with_closest_stations.csv


Function for calculating closest public hospitals

In [18]:
def find_closest_location(house_coords, locations):
    closest_location = None
    min_distance = float('inf')
    closest_location_id = None
    closest_location_name = None
    
    for (location_coords, location_id, location_name) in locations:
        distance = geodesic(house_coords, location_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_location = location_coords
            closest_location_id = location_id
            closest_location_name = location_name
            
    return closest_location, closest_location_id, closest_location_name, min_distance

In [19]:
hospital_file = '../data/raw/vic_hospitals_with_coordinates.csv'
hospitals_df = pd.read_csv(hospital_file)

hospitals_df['geometry'] = hospitals_df['geometry'].apply(eval)

hospital_data = [((row['geometry'][1], row['geometry'][0]), row['Establishment ID'], row['Hospital name']) 
                 for idx, row in hospitals_df.iterrows()]

In [20]:
for idx, row in houses_df.iterrows():
    house_coords = (row['geometry']['latitude'], row['geometry']['longitude'])
    
    # Find closest hospital
    closest_hospital_coords, closest_hospital_id, closest_hospital_name, hospital_distance = find_closest_location(house_coords, hospital_data)
    
    # Update DataFrame with closest hospital data
    houses_df.at[idx, 'closest_hospital_name'] = closest_hospital_name
    houses_df.at[idx, 'closest_hospital_distance_km'] = hospital_distance
    
    # Print progress every 1000 houses
    if (idx + 1) % 1000 == 0:
        print(f"Processed {idx + 1} houses...")

output_file = '../data/curated/houses_with_closest_stations_and_hospital.csv'
houses_df.to_csv(output_file, index=False)

print(f"Data with closest hospitals saved to: {output_file}")

Processed 1000 houses...
Processed 2000 houses...
Processed 3000 houses...
Processed 4000 houses...
Processed 5000 houses...
Processed 6000 houses...
Processed 7000 houses...
Processed 8000 houses...
Processed 9000 houses...
Data with closest hospitals saved to: ../data/curated/houses_with_closest_stations_and_hospital.csv


Function for getting nearest grocery store

In [21]:
grocery_df = pd.read_csv('../data/raw/grocery_markets_victoria_processed.csv')


In [22]:
from shapely.wkt import loads  
def extract_coordinates(geometry):
    point = loads(geometry)  
    return point.y, point.x  


In [23]:
grocery_df['latitude'], grocery_df['longitude'] = zip(*grocery_df['geometry'].apply(extract_coordinates))
grocery_data = list(zip(grocery_df[['latitude', 'longitude']].apply(tuple, axis=1), grocery_df['name']))


In [24]:
def find_closest_location(house_coords, locations):
    closest_location_name = None
    min_distance = float('inf')
    
    for location_coords, location_name in locations:
        distance = geodesic(house_coords, location_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_location_name = location_name
            
    return closest_location_name, min_distance

In [25]:
for idx, row in houses_df.iterrows():
    house_coords = (row['geometry']['latitude'], row['geometry']['longitude'])
    
    closest_grocery_name, grocery_distance = find_closest_location(house_coords, grocery_data)
    
    houses_df.at[idx, 'closest_grocery_name'] = closest_grocery_name
    houses_df.at[idx, 'closest_grocery_distance_km'] = grocery_distance
    
    # Print progress every 1000 houses
    if (idx + 1) % 1000 == 0:
        print(f"Processed {idx + 1} houses...")

output_file = '../data/curated/houses_with_closest_groceries.csv'
houses_df.to_csv(output_file, index=False)

print(f"Data with closest groceries saved to: {output_file}")

Processed 1000 houses...
Processed 2000 houses...
Processed 3000 houses...
Processed 4000 houses...
Processed 5000 houses...
Processed 6000 houses...
Processed 7000 houses...
Processed 8000 houses...
Processed 9000 houses...
Data with closest groceries saved to: ../data/curated/houses_with_closest_groceries.csv
